# Capstone Modeling Script

In [39]:
#Import general data manipulation/plotting packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from scipy import stats

#Import data preprocessing packages
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

#Import train/test and cross validation packages
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFECV
from sklearn.model_selection import cross_val_score

#Import regression packages
import statsmodels
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels import regression
from patsy import dmatrices
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

#Import random forest package
from sklearn.ensemble import RandomForestRegressor

#Import scoring metrics
from sklearn.metrics import mean_squared_error

#allow all columns to be viewed:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
current_path = % pwd
#If current_path is in Scripts folder, 
#go up one level so we can open the data folder
if (current_path.rsplit('\\', 1)[1] == 'Scripts'):
    % cd ..

D:\Coding Projects\Springboard\Springboard_Projects\Capstone


In [3]:
#Import data

#Read in AirBnB dataset that has been merged with summarized Yelp information:
#... (number of businesses, total reviews, and average star rating for businesses w/in .1 and .5 miles)
path1='../Capstone/Data/abb_stat_inf_changes.csv'
abb = pd.read_csv(path1)

print(abb.shape)
abb.head()

(16011, 73)


,id,host_response_time,host_response_rate,host_is_superhost,neighborhood,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,instant_bookable,cancellation_policy,square_feet_notNA,monthly_price_notNA,weekly_price_notNA,bathrooms_notNA,beds_notNA,bedrooms_notNA,security_deposit_notNA,cleaning_fee_notNA,host_response_rate_notNA,reviews_per_month_notNA,neighbourhood_notNA,host_neighbourhood_notNA,neighbourhood_cleansed_notNA,host_response_time_notNA,host_is_superhost_notNA,has_Wifi,has_Heating,has_Essentials,has_Kitchen,has_Smoke_detector,has_Air_conditioning,has_Hangers,has_Washer,has_Dryer,has_Shampoo,has_TV,has_Familykid_friendly,has_Elevator,has_Free_parking_on_premises,has_Internet,has_Gym,has_Cable_TV,has_Paid_parking_off_premises,has_Pool,has_Hot_tub,has_Pets_allowed,has_Breakfast,has_Buzzerwireless_intercom,has_Indoor_fireplace,has_Free_street_parking,has_Wheelchair_accessible,has_Doorman,has_Pets_live_on_this_property,has_Smoking_allowed,host_lives_near_listing,yelp_bus_count_1,yelp_bus_total_reviews_1,yelp_bus_avg_rating_1,yelp_bus_count_5,yelp_bus_total_reviews_5,yelp_bus_avg_rating_5,log_price,Utilization_Rate
0,1419,within an hour,0.0,f,Little Portugal,House,Entire home/apt,6.0,2.0,3.0,4.0,Real Bed,470.0,1000.0,150.0,1.0,0.0,4.0,f,strict_14_with_grace_period,True,False,True,True,True,True,True,True,False,True,True,True,True,False,True,True,False,False,True,False,True,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,1.0,66.0,3.50,116.0,8634.0,3.758621,6.152733,0.013053
1,10314,within an hour,0.0,f,Riverdale,House,Private room,2.0,1.0,1.0,1.0,Real Bed,69.0,0.0,0.0,2.0,20.0,1.0,f,moderate,False,True,True,True,True,True,False,False,False,True,True,True,True,False,True,True,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,4.234107,0.042601
2,12604,within an hour,0.0,f,The Annex,House,Private room,1.0,1.5,1.0,1.0,Pull-out Sofa,65.0,130.0,26.0,1.0,20.0,1.0,f,moderate,True,True,True,True,True,True,True,True,False,False,True,True,True,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,NaN,NaN,NaN,11.0,1631.0,3.045455,4.174387,0.000866
3,17936,within an hour,100.0,t,Kensington Market,Apartment,Private room,4.0,1.0,1.0,2.0,Real Bed,99.0,300.0,80.0,1.0,20.0,2.0,f,strict_14_with_grace_period,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,True,False,True,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,20.0,1440.0,3.65,160.0,11434.0,3.715625,4.595120,0.105691
4,23691,within an hour,100.0,t,Wychwood,House,Private room,2.0,1.0,1.0,1.0,Real Bed,70.0,0.0,0.0,2.0,25.0,1.0,t,strict_14_with_grace_period,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,True,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,NaN,NaN,NaN,11.0,147.0,3.363636,4.248495,0.092562


## Drop Unecessary Variables

### Drop Uneeded Amenities
Earlier in the Statistical Inference script I ran a series of t-tests to determine whether each Amenity has a statistically significant relationship to Price and/or Utilization Rate.

I plan to drop Amenities that did not have a statistically significant relationship. My desired p-value to determine statistical significance is 0.05. However I know that when making multiple comparisons, it is often very easy to have tests that pass the threshold purely by chance. The Bonferroni correction is a simple way to avoid this issue. 

The number of comparisons being made is 29, so I know my actual p-value threshold needs to be (0.05/29=) 0.0017. I will drop any amenities that did not pass that threshold. 

Note: There were differences in which Amenities were statistically significant against Price vs. Utilization Rate. At this point I will create two datasets, one to predict Price, the other Utilization Rate, so I can keep the variables with the best relationship to each outcome. 

In [4]:
#Create copies of dataset
abb_Price_predict = abb.copy()
abb_Util_predict = abb.copy()

In [5]:
#Drop amenity variables with a p-value greater than 0.0017 from Price prediction dataset
drop_cols = ['has_Wifi', 'has_Doorman', 'has_Indoor_fireplace', 'has_Familykid_friendly', 'has_Wheelchair_accessible',
            'has_Hot_tub', 'has_Hangers', 'has_Buzzerwireless_intercom', 'has_Shampoo']

abb_Price_predict.drop(columns=drop_cols, axis=1, inplace=True)

abb_Price_predict.shape

(16011, 64)

In [6]:
#Drop amenity variables with a p-value greater than 0.0017 from Utilization_Rate prediction dataset
drop_cols = ['has_Hangers', 'has_Smoking_allowed', 'has_Pool', 'has_Hot_tub', 'has_Air_conditioning', 'has_Elevator',
            'has_Gym', 'has_Familykid_friendly', 'has_Free_parking_on_premises', 'has_Smoke_detector', 'has_Wheelchair_accessible',
            'has_Pets_allowed', 'has_Indoor_fireplace', 'has_Paid_parking_off_premises', 'has_Breakfast', 'has_Doorman',
            'has_Essentials', 'has_Free_street_parking', 'has_Pets_live_on_this_property', 'has_Shampoo', 'has_Buzzerwireless_intercom']

abb_Util_predict.drop(columns=drop_cols, axis=1, inplace=True)

abb_Util_predict.shape

(16011, 52)

### Drop Uneeded "NA" Flag Variables

Now I will do the same for the \_notNA variables, which are flag variables I created during data wrangling to tell me if a value was missing for a given variable. I did a series of t-tests on these against Price and Utilization to see which had a statistically significant relationship. 

This time the number of comparisons was 10, so the p-value threshold they need to pass for significance is (0.05/10=) 0.005.

Note: Technically there were 15 \_notNA variables tested in the Statistical Inference script, but this was a mistake. 5 of those had no NA values (likely the records with missing values were dropped over time for those) so they had no actual data to test. Those variables will be dropped below as well. 

In [7]:
#Drop _notNA variables with a p-value greater than 0.0017 from Price prediction dataset
drop_cols = ['bathrooms_notNA', 'beds_notNA', 'bedrooms_notNA', 'neighbourhood_cleansed_notNA', 'host_is_superhost_notNA',
            'monthly_price_notNA', 'weekly_price_notNA', 'neighbourhood_notNA', 'host_response_rate_notNA', 'host_response_time_notNA',
            'host_neighbourhood_notNA', 'square_feet_notNA']

abb_Price_predict.drop(columns=drop_cols, axis=1, inplace=True)

abb_Price_predict.shape

(16011, 52)

In [8]:
#Drop _notNA variables with a p-value greater than 0.0017 from Utilization Rate prediction dataset
drop_cols = ['bathrooms_notNA', 'beds_notNA', 'bedrooms_notNA', 'neighbourhood_cleansed_notNA', 'host_is_superhost_notNA',
            'square_feet_notNA', 'weekly_price_notNA', 'host_neighbourhood_notNA', 'monthly_price_notNA']

abb_Util_predict.drop(columns=drop_cols, axis=1, inplace=True)

abb_Util_predict.shape

(16011, 43)

### Drop Yelp Business Metrics w/out Statistical Significance, Create Flag Variables for Having a Business w/in .1 and .5 Miles

Next I will drop the Yelp Business metrics that did not show a statistically significant correlation with Price/Utilization (or if the correlation was extremely low). 

As a reminder, the Yelp Business metrics are Number of Businesses, Total Reviews, and Avg. Star Rating. These were calculated for all businesses within .1 and .5 miles of each AirBnB listing, for 6 variables total. 

All three of the metrics for busineses within .1 mile had a very low correlation with both Price and Utilization. I will drop all three of the .1 mile metrics. However when doing a t-test comparing the Price of listings with at least one business within .1 mile and those that did not, there was a large and statistically significant price difference (same for those with/without businesses within .5 mile). So before dropping the .1 mile metrics I will create a flag variable that is True if the AirBnB listing has a business within .1 (and .5) miles.

In [9]:
#Create flag variable indicating if any business was within .1, .5 miles of an AirBnB listing
abb_Price_predict['Bus_in_pt_1'] = pd.notna(abb_Price_predict['yelp_bus_count_1'])
abb_Price_predict['Bus_in_pt_5'] = pd.notna(abb_Price_predict['yelp_bus_count_5'])

abb_Util_predict['Bus_in_pt_1'] = pd.notna(abb_Util_predict['yelp_bus_count_1'])
abb_Util_predict['Bus_in_pt_5'] = pd.notna(abb_Util_predict['yelp_bus_count_5'])

In [10]:
#Drop .1 mile Yelp Business variables
drop_cols = ['yelp_bus_count_1', 'yelp_bus_total_reviews_1', 'yelp_bus_avg_rating_1']

abb_Price_predict.drop(columns=drop_cols, axis=1, inplace=True)
abb_Util_predict.drop(columns=drop_cols, axis=1, inplace=True)

print(abb_Price_predict.shape)
print(abb_Util_predict.shape)

(16011, 51)
(16011, 42)


The .5 mile Yelp Business metrics, specifically Number of Businesses and Total Reviews, did show both a practical correlation coefficient and statistical significance when compared to Price, so those variables will be kept in the Price prediction dataset. The Avg. Rating was not practically significant against Price, so it will be dropped.

All three .5 mile Yelp Business metrics were not practically significant against Utilization Rate and will be dropped. 

In [11]:
#Drop .5 mile Yelp Business variables from price prediction dataset
drop_cols = ['yelp_bus_avg_rating_5']

abb_Price_predict.drop(columns=drop_cols, axis=1, inplace=True)

print(abb_Price_predict.shape)

(16011, 50)


In [12]:
#Drop .5 mile Yelp Business variables from utilization prediction dataset
drop_cols = ['yelp_bus_count_5', 'yelp_bus_total_reviews_5', 'yelp_bus_avg_rating_5']

abb_Util_predict.drop(columns=drop_cols, axis=1, inplace=True)

print(abb_Util_predict.shape)

(16011, 39)


As a final step I will replace NaN values in the Yelp metrics with 0 to ensure the modeling algorithms do not drop those records.

In [13]:
abb_Price_predict[['yelp_bus_count_5', 'yelp_bus_total_reviews_5']] = abb_Price_predict[['yelp_bus_count_5', 'yelp_bus_total_reviews_5']].fillna(0.0)

## Predicting Price

### Produce Baseline Price Prediction Model
First I will create a simple baseline linear regression model predicting raw price (not log price) with all of the predictor variables. This will serve as my baseline model to compare future models against. Before doing that I need to convert my categorical predictor variables into binary flag variables, then split my data into train test splits. I will also scale my numeric variables, but I will handle that within the modeling pipeline. Scaling is not necessarily a requirement for standard linear regression, but I know I will use Ridge, Lasso, Random Forest, and possibly other model types later on that will require scaled numeric variables. 

In [14]:
#Perform get_dummies to one-hot encode categorical variables
cat_cols = abb_Price_predict[['host_response_time', 'neighborhood', 'host_is_superhost', 'room_type', 'bed_type',
                            'instant_bookable', 'cancellation_policy', 'property_type']].columns

abb_Price_predict_expanded = abb_Price_predict.copy()
abb_Price_predict_expanded = pd.get_dummies(abb_Price_predict, sparse=True, columns=cat_cols)

#Confirm only numeric variables remain
abb_Price_predict_expanded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16011 entries, 0 to 16010
Columns: 115 entries, id to property_type_Townhouse
dtypes: bool(26), float64(15), int64(1), uint8(73)
memory usage: 2.5 MB


In [17]:
#Create X and y
X_vars = abb_Price_predict_expanded.columns.difference(['id', 'price', 'Utilization_Rate', 'log_price'])
X = abb_Price_predict_expanded[X_vars]
y = abb_Price_predict_expanded[['price', 'log_price', 'Utilization_Rate']]

In [18]:
#Create train-test split
X_train, X_test, y_train,  y_test = train_test_split(X, y, 
                                                     test_size = 0.3, 
                                                     random_state=17)

Next I will create a pipeline that handles scaling and modeling in one step. I will use MinMax instead of Standard scaler because I know most of my numeric variables do not follow a normal distribution, and are in fact skewed right (long tail to the right). 

In [27]:
#Initialize MinMaxScaler
scaler = MinMaxScaler()
#Initialize Linear Regression
regression = LinearRegression()
#Create pipeline
pipeline_ols = Pipeline([("Scaler", scaler), ("Regression", regression)])

In [134]:
pipeline_ols.fit(X_train, y_train['price'])

Pipeline(memory=None,
     steps=[('Scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('Regression', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False))])

In [47]:
#Calculate R^2 on train and test sets
ols_r2_train = pipeline_ols.score(X_train, y_train['price'])
ols_r2_test = pipeline_ols.score(X_test, y_test['price'])

In [46]:
#Calculate Mean Squared Error on test and train sets
x_train_pred = pipeline_ols.predict(X_train)
x_test_pred = pipeline_ols.predict(X_test)

mse_train = mean_squared_error(y_train['price'], x_train_pred)
mse_test = mean_squared_error(y_test['price'], x_test_pred)

In [49]:
scores_df = pd.DataFrame([{'Model':'OLS_RawPrice', 'R2_train':ols_r2_train, 'R2_test':ols_r2_test, 
                          'MSE_train':mse_train, 'MSE_test':mse_test}])

In [51]:
scores_df[['Model', 'R2_train', 'R2_test', 'MSE_train', 'MSE_test']]

,Model,R2_train,R2_test,MSE_train,MSE_test
0,OLS_RawPrice,0.492882,0.500269,4468.043447,4623.699172


In [138]:
coefficients = pd.DataFrame({"Feature":X_train.columns, 
                             "Coefficients_RawPrice":np.transpose(pipeline_ols.named_steps['Regression'].coef_)})

coefficients[coefficients['Coefficients_RawPrice']!=0].count()

Feature                  111
Coefficients_RawPrice    111
dtype: int64

In [139]:
coefficients

,Feature,Coefficients_RawPrice
0,Bus_in_pt_1,3.092931e+00
1,Bus_in_pt_5,3.839140e+00
2,accommodates,4.298917e+01
3,bathrooms,2.908909e+01
4,bed_type_Airbed,-1.562154e+12
5,bed_type_Couch,-1.562154e+12
6,bed_type_Futon,-1.562154e+12
7,bed_type_Pull-out Sofa,-1.562154e+12
8,bed_type_Real Bed,-1.562154e+12
9,bedrooms,6.245398e+01


My model has many input variables, 111 with a coefficient not equal to 0 (which is all of the variables). Feature selection will need to be performed to fine-tune this model. 

Interestingly the R2 value is decent and actually improves from train to test (0.49 to 0.5). Mean Squared Error slightly increases from train to test. Higher is better for R2, lower is better for MSE. 

Next I will try running the same model on 

### Fit Model on Log Price using Trimmed Variables
Next I will test running the model on the log of Price, to see if this improves anything. 

In [140]:
pipeline_ols.fit(X_train, y_train['log_price'])

Pipeline(memory=None,
     steps=[('Scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('Regression', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False))])

In [128]:
#Calculate R^2 on train and test sets
ols_r2_train = pipeline_ols.score(X_train, y_train['log_price'])
ols_r2_test = pipeline_ols.score(X_test, y_test['log_price'])

In [129]:
#Calculate Mean Squared Error on test and train sets
x_train_pred = pipeline_ols.predict(X_train)
x_test_pred = pipeline_ols.predict(X_test)

mse_train = mean_squared_error(y_train['log_price'], x_train_pred)
mse_test = mean_squared_error(y_test['log_price'], x_test_pred)

In [130]:
scores_df = scores_df.append(pd.DataFrame([{'Model':'OLS_LogPrice', 'R2_train':ols_r2_train, 'R2_test':ols_r2_test, 
                          'MSE_train':mse_train, 'MSE_test':mse_test}]))

In [131]:
scores_df[['Model', 'R2_train', 'R2_test', 'MSE_train', 'MSE_test']]

,Model,R2_train,R2_test,MSE_train,MSE_test
0,OLS_RawPrice,0.492882,0.500269,4468.043447,4623.699172
0,OLS_LogPrice,0.639502,0.633953,0.150438,0.157333


Modeling on Log of Price provides a much improved R2 score, from 0.5 to 0.63 on the testing set. MSE also decreases but this is difficult to compare as Log Price changes the relative size of errors. However going forward I will use Log Price in my models and will be able to compare the MSE from this OLS to future models (Lasso, Ridge, Random Forest).

In [141]:
coefficients_log = pd.DataFrame({"Feature":X_train.columns, 
                             "Coefficients_LogPrice":np.transpose(pipeline_ols.named_steps['Regression'].coef_)})

coefficients_log[coefficients_log['Coefficients_LogPrice']!=0].count()

Feature                  111
Coefficients_LogPrice    111
dtype: int64

In [142]:
coefficients = coefficients.merge(coefficients_log)
coefficients

,Feature,Coefficients_RawPrice,Coefficients_LogPrice
0,Bus_in_pt_1,3.092931e+00,2.177700e-02
1,Bus_in_pt_5,3.839140e+00,2.896165e-02
2,accommodates,4.298917e+01,3.814274e-01
3,bathrooms,2.908909e+01,8.213780e-02
4,bed_type_Airbed,-1.562154e+12,-1.703562e+10
5,bed_type_Couch,-1.562154e+12,-1.703562e+10
6,bed_type_Futon,-1.562154e+12,-1.703562e+10
7,bed_type_Pull-out Sofa,-1.562154e+12,-1.703562e+10
8,bed_type_Real Bed,-1.562154e+12,-1.703562e+10
9,bedrooms,6.245398e+01,4.046383e-01


The model still has too many predictors, so next I will perform feature selection steps. 

### Perform Feature Selection Procedures

#### Fit LASSO Model

I would like to reduce the number of predictor variables I am feeding into the model to reduce complexity. To reach that goal I will try LASSO next, because I know it can shrink many predictors to 0. 

I will use GridSearchCV to iterate through different values of alpha and choose the one that reduces complexity while not impacting MSE too significantly. 

In [169]:
#Initialize Lasso
lasso = Lasso()
#Create pipeline
pipeline_lasso = Pipeline([("Scaler", scaler), ("Lasso", lasso)])
#Initialize GridSearchCV
alphas = np.arange(0.01, 1, 0.01)
search_space = [{"Lasso__alpha" : alphas}]
gridsearch_lasso = GridSearchCV(pipeline_lasso, search_space, cv=5, verbose=0, n_jobs=-1)

In [170]:
best_lasso = gridsearch_lasso.fit(X_train, y_train['log_price'])

In [171]:
best_lasso.best_estimator_.get_params()['Lasso__alpha']

0.01

In [172]:
#Calculate R^2 on train and test sets
lasso_r2_train = best_lasso.best_estimator_.score(X_train, y_train['log_price'])
lasso_r2_test = best_lasso.best_estimator_.score(X_test, y_test['log_price'])

In [173]:
#Calculate Mean Squared Error on test and train sets
x_train_pred = best_lasso.best_estimator_.predict(X_train)
x_test_pred = best_lasso.best_estimator_.predict(X_test)

mse_train = mean_squared_error(y_train['log_price'], x_train_pred)
mse_test = mean_squared_error(y_test['log_price'], x_test_pred)

In [177]:
scores_df = scores_df.append(pd.DataFrame([{'Model':'Lasso_LogPrice', 'R2_train':lasso_r2_train, 'R2_test':lasso_r2_test, 
                          'MSE_train':mse_train, 'MSE_test':mse_test}]))

In [178]:
scores_df[['Model', 'R2_train', 'R2_test', 'MSE_train', 'MSE_test']]

,Model,R2_train,R2_test,MSE_train,MSE_test
0,OLS_RawPrice,0.492882,0.500269,4468.043447,4623.699172
0,OLS_LogPrice,0.639502,0.633953,0.150438,0.157333
0,Lasso_LogPrice,0.575324,0.572254,0.177219,0.183852


In [181]:
coefficients_Lasso = pd.DataFrame({"Feature":X_train.columns, 
                             "Coefficients_LogPrice_Lasso":np.transpose(best_lasso.best_estimator_.named_steps.Lasso.coef_)})

coefficients_Lasso[coefficients_Lasso['Coefficients_LogPrice_Lasso']!=0].count()

Feature                        16
Coefficients_LogPrice_Lasso    16
dtype: int64

In [182]:
coefficients = coefficients.merge(coefficients_Lasso)
coefficients

,Feature,Coefficients_RawPrice,Coefficients_LogPrice,Coefficients_LogPrice_Lasso
0,Bus_in_pt_1,3.092931e+00,2.177700e-02,0.085549
1,Bus_in_pt_5,3.839140e+00,2.896165e-02,0.000000
2,accommodates,4.298917e+01,3.814274e-01,0.453721
3,bathrooms,2.908909e+01,8.213780e-02,0.100165
4,bed_type_Airbed,-1.562154e+12,-1.703562e+10,-0.000000
5,bed_type_Couch,-1.562154e+12,-1.703562e+10,-0.000000
6,bed_type_Futon,-1.562154e+12,-1.703562e+10,-0.000000
7,bed_type_Pull-out Sofa,-1.562154e+12,-1.703562e+10,-0.000000
8,bed_type_Real Bed,-1.562154e+12,-1.703562e+10,0.000000
9,bedrooms,6.245398e+01,4.046383e-01,0.225400


In [227]:
lasso_vars = list(coefficients[coefficients['Coefficients_LogPrice_Lasso'] != 0]['Feature'])

In [228]:
lasso_vars

['Bus_in_pt_1',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'cleaning_fee',
 'has_Air_conditioning',
 'has_Heating',
 'has_Kitchen',
 'has_TV',
 'neighborhood_Downtown Core',
 'neighborhood_Entertainment District',
 'neighborhood_Scarborough',
 'property_type_Apartment',
 'property_type_Condominium',
 'reviews_per_month_notNA',
 'room_type_Entire home/apt']

Lasso through GridSearchCV chose 0.01 as the best value for Alpha, the lowest possible value from the search space I gave it. This still dropped my number of variables from 111 to 16, which is exactly what I was looking for. And the resulting model performs only slightly worse than the prior OLS on all variables predicting Log Price. R2 dropped from 0.634 to 0.572, and MSE dropped from 0.157 to 0.183, on the test set. But I believe I can live with the performance decrease for the improved interpretability of the model. 

I am not done yet - next I will try using Recursive Feature Elimination.

#### Use Recursive Feature Elimination to determine relevant variables

In [188]:
regression = LinearRegression()

In [211]:
rfecv = RFECV(estimator=regression, step=1, scoring='neg_mean_squared_error')

In [212]:
X_train_scaled = scaler.fit_transform(X_train)

In [213]:
rfecv.fit(X_train_scaled, y_train['log_price'])

RFECV(cv=None,
   estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False),
   n_jobs=1, scoring='neg_mean_squared_error', step=1, verbose=0)

In [214]:
rfecv.n_features_

108

In [198]:
rfecv.ranking_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1,
       1])

Recursive Feature Elimination only removes 3 variables from my model which is not very helpful. I will not use this output going forward. 

Next I will try running Ridge Regression, first with all variables, then with just the variables identified by Lasso.

### Fit Ridge Regression Models

In [229]:
#Initialize Ridge
ridge = Ridge()
#Create pipeline
pipeline_ridge = Pipeline([("Scaler", scaler), ("Ridge", ridge)])
#Initialize GridSearchCV
alphas = np.arange(0.01, 5, 0.01)
search_space = [{"Ridge__alpha" : alphas}]
gridsearch_ridge = GridSearchCV(pipeline_ridge, search_space, cv=5, verbose=0, n_jobs=-1)

#### Fit with all variables

In [232]:
best_ridge_AllVars = gridsearch_ridge.fit(X_train, y_train['log_price'])

In [233]:
best_ridge_AllVars.best_estimator_.get_params()['Ridge__alpha']

4.99

In [234]:
#Calculate R^2 on train and test sets
ridge_r2_train = best_ridge_AllVars.best_estimator_.score(X_train, y_train['log_price'])
ridge_r2_test = best_ridge_AllVars.best_estimator_.score(X_test, y_test['log_price'])

In [235]:
#Calculate Mean Squared Error on test and train sets
x_train_pred = best_ridge_AllVars.best_estimator_.predict(X_train)
x_test_pred = best_ridge_AllVars.best_estimator_.predict(X_test)

mse_train = mean_squared_error(y_train['log_price'], x_train_pred)
mse_test = mean_squared_error(y_test['log_price'], x_test_pred)

In [236]:
scores_df = scores_df.append(pd.DataFrame([{'Model':'Ridge_LogPrice_AllVars', 'R2_train':ridge_r2_train, 'R2_test':ridge_r2_test, 
                          'MSE_train':mse_train, 'MSE_test':mse_test}]))

#### Fit with Lasso Variables

In [238]:
best_ridge_LassoVars = gridsearch_ridge.fit(X_train[lasso_vars], y_train['log_price'])

In [239]:
best_ridge_LassoVars.best_estimator_.get_params()['Ridge__alpha']

4.99

In [240]:
#Calculate R^2 on train and test sets
ridge_r2_train = best_ridge_LassoVars.best_estimator_.score(X_train[lasso_vars], y_train['log_price'])
ridge_r2_test = best_ridge_LassoVars.best_estimator_.score(X_test[lasso_vars], y_test['log_price'])

In [241]:
#Calculate Mean Squared Error on test and train sets
x_train_pred = best_ridge_LassoVars.best_estimator_.predict(X_train[lasso_vars])
x_test_pred = best_ridge_LassoVars.best_estimator_.predict(X_test[lasso_vars])

mse_train = mean_squared_error(y_train['log_price'], x_train_pred)
mse_test = mean_squared_error(y_test['log_price'], x_test_pred)

In [242]:
scores_df = scores_df.append(pd.DataFrame([{'Model':'Ridge_LogPrice_LassoVars', 'R2_train':ridge_r2_train, 'R2_test':ridge_r2_test, 
                          'MSE_train':mse_train, 'MSE_test':mse_test}]))

In [243]:
scores_df[['Model', 'R2_train', 'R2_test', 'MSE_train', 'MSE_test']]

,Model,R2_train,R2_test,MSE_train,MSE_test
0,OLS_RawPrice,0.492882,0.500269,4468.043447,4623.699172
0,OLS_LogPrice,0.639502,0.633953,0.150438,0.157333
0,Lasso_LogPrice,0.575324,0.572254,0.177219,0.183852
0,Ridge_LogPrice_AllVars,0.639353,0.634300,0.150500,0.157184
0,Ridge_LogPrice_LassoVars,0.594658,0.593120,0.169151,0.174884


Ridge Regression predicting Log Price with only the variables identified by Lasso performs well, only slightly worse than Ridge with all variables and much easier to interpret. For now I will consider Ridge Regression with Lasso Variables as my best model. 

### Fit Random Forest

Next I plan to use Random Forest as another model to compare against. I will use GridSearchCV again to test multiple iterations of the number of estimators (the number of trees to build), max tree depth, minimum samples to split a node, minimum samples required at each leaf, and whether to use bootstrapping or not to select samples. 

In [247]:
#Initialize random forest regressor
rfr = RandomForestRegressor(n_jobs=1, random_state=17)
#Create pipeline
pipeline_rfr = Pipeline([("Scaler", scaler), ("rfr", rfr)])
#Initialize GridSearchCV
n_estimators = np.arange(10, 500, 10)
max_depth = np.arange(10, 100, 10)
min_samples_split = [2, 10, 100]
min_samples_leaf = [1, 2, 4, 50]
bootstrap = [True, False]

search_space_rfr = [{"rfr__n_estimators" : n_estimators,
                "rfr__max_depth" : max_depth,
                "rfr__min_samples_split" : min_samples_split,
                "rfr__min_samples_leaf" : min_samples_leaf,
                "rfr__bootstrap" : bootstrap}]

gridsearch_rfr = GridSearchCV(pipeline_rfr, search_space_rfr, cv=5, verbose=0, n_jobs=-1)

In [248]:
#Fit random forest on all variables
rfr_all_vars = gridsearch_rfr.fit(X_train, y_train['log_price'])

rfr_lasso_vars = gridsearch_rfr.fit(X_train[lasso_vars], y_train['log_price'])

KeyboardInterrupt: 

In [ ]:
#Pickle models (cell above takes 3+ hours to complete)
import pickle

filename1 = 'rfr_all_vars_model.pkl'
filename2 = 'rfr_lasso_vars_model.pkl'

In [ ]:
with open(filename1, 'wb') as file:
    pickle.dump(rfr_all_vars, file)

with open(filename2, 'wb') as file:
    pickle.dump(rfr_lasso_vars, file)

In [ ]:
#Import models from pickle files - do not run models again, takes many hours!
with open(filename1, 'rb') as file:
    rfr_all_vars = pickle.load(file)
    
with open(filename2, 'rb') as file:
    rfr_lasso_vars = pickle.load(file)

In [ ]:
rfr_all_vars.best_estimator_.get_params()

In [239]:
rfr_lasso_vars.best_estimator_.get_params()

4.99

In [240]:
#Calculate R^2 on train and test sets
ridge_r2_train = best_ridge_LassoVars.best_estimator_.score(X_train[lasso_vars], y_train['log_price'])
ridge_r2_test = best_ridge_LassoVars.best_estimator_.score(X_test[lasso_vars], y_test['log_price'])

In [241]:
#Calculate Mean Squared Error on test and train sets
x_train_pred = best_ridge_LassoVars.best_estimator_.predict(X_train[lasso_vars])
x_test_pred = best_ridge_LassoVars.best_estimator_.predict(X_test[lasso_vars])

mse_train = mean_squared_error(y_train['log_price'], x_train_pred)
mse_test = mean_squared_error(y_test['log_price'], x_test_pred)

In [242]:
scores_df = scores_df.append(pd.DataFrame([{'Model':'Ridge_LogPrice_LassoVars', 'R2_train':ridge_r2_train, 'R2_test':ridge_r2_test, 
                          'MSE_train':mse_train, 'MSE_test':mse_test}]))

In [243]:
scores_df[['Model', 'R2_train', 'R2_test', 'MSE_train', 'MSE_test']]

,Model,R2_train,R2_test,MSE_train,MSE_test
0,OLS_RawPrice,0.492882,0.500269,4468.043447,4623.699172
0,OLS_LogPrice,0.639502,0.633953,0.150438,0.157333
0,Lasso_LogPrice,0.575324,0.572254,0.177219,0.183852
0,Ridge_LogPrice_AllVars,0.639353,0.634300,0.150500,0.157184
0,Ridge_LogPrice_LassoVars,0.594658,0.593120,0.169151,0.174884


In [ ]:
features = list(X_vars)
importances = rfc.feature_importances_
indices = np.argsort(importances)

In [ ]:
fig=plt.figure(figsize=(10, 25), dpi=120)
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

In [ ]:
features_df = pd.DataFrame(features)
features_df.rename(columns={0:'Parameter'}, inplace=True)

In [ ]:
importances_df = pd.DataFrame(importances)
importances_df.rename(columns={0:'RF_Imp'}, inplace=True)

In [ ]:
rf_vars = features_df.join(importances_df)

In [ ]:
rf_vars = rf_vars[rf_vars['RF_Imp'] > 0.01]
rf_vars

Setting the threshold at 0.01 importance score gives 13 variables identified as "important" by the random forest model. 

I will use one more method to determine the best features to use in the model: Recursive Feature Elimination using scikit-learn's RFECV function. 

### Test each subset of variables using Ridge Regression and Cross Validation

Now that I have created subsets of relevant variables using Lasso, Random Forest, and Recursive Feature Elimination, it is time to test these subsets in an actual model. 

In [ ]:
#Create train-test split
X_train, X_test, y_train,  y_test = train_test_split(X, y, 
                                                     test_size = 0.3, 
                                                     random_state=17)

In [ ]:
#Initialize Model Parameters, Ridge Model, and GridSearchCV
param_grid = {'alpha':np.arange(0, 1, 0.05)}
ridge_model = linear_model.Ridge(random_state=17)
ridge_cv = GridSearchCV(ridge_model, param_grid, cv=10, scoring='neg_mean_squared_error')

In [ ]:
#Fit ridge with all predictors
ridge_cv.fit(X_train, y_train)

In [ ]:
#Extract scoring values
alpha1 = ridge_cv.best_params_
train_mse1 = ridge_cv.best_score_
test_mse1 = ridge_cv.score(X_test, y_test)

In [ ]:
#Add scoring values to a data frame (grid_results) for easier comparison
grid_results = pd.DataFrame(columns=['Vars', 'Alpha', 'train_mse', 'test_mse'])
grid_results = grid_results.append([{'Vars':'All', 'Alpha':alpha1, 
                                     'train_mse':train_mse1, 'test_mse':test_mse1}])

In [ ]:
#Initialize Lasso predictor variables
lasso_predictors = ['Bus_in_pt_1', 'Bus_in_pt_5', 'has_Air_conditioning', 'has_TV', 'neighborhood_Entertainment District',
                    'neighborhood_Scarborough', 'property_type_Condominium', 'reviews_per_month_notNA',
                   'room_type_Private room', 'accommodates', 'bedrooms', 'cleaning_fee', 'host_response_rate', 'minimum_nights',
                   'security_deposit', 'yelp_bus_count_5', 'yelp_bus_total_reviews_5']

In [ ]:
#Fit gridsearch with lasso predictors
ridge_cv.fit(X_train[lasso_predictors], y_train)

#Add scores to grid_results
alpha2 = ridge_cv.best_params_
train_mse2 = ridge_cv.best_score_
test_mse2 = ridge_cv.score(X_test[lasso_predictors], y_test)

grid_results = grid_results.append([{'Vars':'Lasso', 'Alpha':alpha2, 
                                     'train_mse':train_mse2, 'test_mse':test_mse2}])

In [ ]:
rf_predictors = ['accommodates', 'bathrooms', 'bedrooms', 'cleaning_fee', 'extra_people', 'host_response_rate',
                'minimum_nights', 'property_type_Condominium', 'reviews_per_month_notNA', 'room_type_Entire home/apt',
                'security_deposit', 'yelp_bus_count_5', 'yelp_bus_total_reviews_5']

In [ ]:
#Fit gridsearch with lasso predictors
ridge_cv.fit(X_train[rf_predictors], y_train)

#Add scores to grid_results
alpha3 = ridge_cv.best_params_
train_mse3 = ridge_cv.best_score_
test_mse3 = ridge_cv.score(X_test[rf_predictors], y_test)

grid_results = grid_results.append([{'Vars':'RandomForest', 'Alpha':alpha3, 
                                     'train_mse':train_mse3, 'test_mse':test_mse3}])

In [ ]:
grid_results

#### Check Variance Inflation Factors for Multi-Collinearity

In [ ]:
#X = abb_Price_predict.drop(['id', 'price', 'Utilization_Rate', 'log_price'], axis=1)
#features = "+".join(abb_Price_predict.columns - [['id', 'price', 'Utilization_Rate', 'log_price']])
y, X = dmatrices('price~' + X_cols, abb_Price_predict, return_type='dataframe')

In [ ]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns

In [ ]:
vif.round(1)

## Predicting Utilization Rate

### Produce Baseline Utilization Rate Prediction Model

#### First Calculate Price vs. Predicted Price Delta

#### Fit Ridge Regression Model

Next I will run Ridge regression on the dataset, but first I need to find the optimal value for Alpha. I will loop through values of alpha from 0 to 1, stepping by 0.05, and look for the alpha that only slightly decreases Adjusted R-Squared.

In [ ]:
model = ols(formula_log, abb_Price_predict)

In [ ]:
frames = []
for n in np.arange(0, 1, 0.05).tolist():
    results_fr = model.fit_regularized(L1_wt=0, alpha=n, start_params=base_m_log.params)
    results_fr_fit = sm.regression.linear_model.OLSResults(model, 
                                                        results_fr.params
                                                       )
    frames.append(np.append(n, results_fr_fit.rsquared_adj))

In [ ]:
frames = pd.DataFrame(frames).rename(columns={0:'Alpha', 1:"Adj. R-Squared"})

In [ ]:
fig=plt.figure(figsize=(10, 5), dpi=120)
sns.lineplot(x='Alpha', y='Adj. R-Squared', data=frames,
            markers=True, dashes=True)
plt.title("Adj. R-Squared vs. Alpha")
plt.show()

The line decreases fairly steadily but there is a slight "elbow" at alpha = 0.05, so I will choose that as my alpha for Ridge Regression.

In [ ]:
base_m_Ridge = ols(formula_log, abb_Price_predict).fit_regularized(alpha=0.05, L1_wt=0)

In [ ]:
ridge_params = pd.DataFrame(base_m_Ridge.params)

In [ ]:
parameters = parameters.join(ridge_params)

In [ ]:
parameters.rename(columns={0:'Ridge Model'}, inplace=True)
parameters